In [ ]:
import cv2
import numpy as np
import os

In [ ]:
file_dir = r'E:\Machine Learning\findContours'
save_dir = r'E:\Machine Learning\cannyfile'

In [ ]:
#定义边缘检测函数
def edge(img):
    #高斯模糊,降低噪声
    blurred = cv2.GaussianBlur(img, (3, 3), 0)
    #灰度图像
    gray = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)
    #图像梯度
    xgrad = cv2.Sobel(gray, cv2.CV_16SC1, 1, 0)
    ygrad = cv2.Sobel(gray, cv2.CV_16SC1, 0, 1)
    #计算边缘，50和150参数必须符合1：3或者1：2，用粗筛判定低阈值
    edge_output = cv2.Canny(xgrad, ygrad, 35, 105)
    return edge_output

In [ ]:
#产生边缘图像-批量
def create_object_image():
    i = 0
    for imgfile in os.listdir(file_dir):   
        img_info = os.path.join(file_dir, imgfile)
        img_array = cv2.imread(img_info)
        edgeSav = edge(img_array)
        save_dir = r'E:\Machine Learning\cannyfile\{}'.format(imgfile)
        cv2.imwrite(save_dir, edgeSav)
        i += 1
    return
create_object_image() 

In [ ]:
#产生轮廓提取图像-批量
def Ref_Process():
    i = 0
    for imgfile in os.listdir(save_dir):
        img_info = os.path.join(save_dir, imgfile)
        img_array = cv2.imread(img_info)
        rawImg_info = os.path.join(file_dir, imgfile)
        rawImg_array = cv2.imread(rawImg_info)
                
        blurred = cv2.GaussianBlur(img_array, (3, 3), 0)
        gray = cv2.cvtColor(blurred, cv2.COLOR_RGB2GRAY)
        ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV)
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
        image = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
        
        kernelX = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 1))
        kernelY = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 9))
        image = cv2.dilate(image, kernelX)
        image = cv2.erode(image, kernelX)
        image = cv2.erode(image, kernelY)
        image = cv2.dilate(image, kernelY)
        
        image = cv2.GaussianBlur(image, (11,11), 0)
        
        contours, hierarchy = cv2.findContours(image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE) 
        c = sorted(contours, key=cv2.contourArea, reverse=True)[1]
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect))
        image = cv2.drawContours(rawImg_array,  [box], -1, (0, 0, 255), 3)
         
        save_dir2 = r'E:\Machine Learning\Refine_file\{}'.format(imgfile)
        cv2.imwrite(save_dir2, image)
        i += 1
    return
Ref_Process()